# Data Exploration

In [ ]:
# 下载数据集

import numpy as np
import pandas as pd
import cv2
from pathlib import Path
from colorama import Fore, Back, Style
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, Image


ROOT_DIR = Path("/kaggle/input/tensorflow-great-barrier-reef")

TRAIN_CSV = ROOT_DIR / "train.csv"
TRAIN_DF = pd.read_csv(TRAIN_CSV)

TEST_CSV = ROOT_DIR / "test.csv"
TEST_DF = pd.read_csv(TEST_CSV)

list(ROOT_DIR.iterdir())

## 用常用的东西来扩充

In [ ]:
import json

# 计数检测
if "detection_count" not in TRAIN_DF.columns:
    det_counts = TRAIN_DF.apply(lambda row: len(eval(row.annotations)), axis=1)
    TRAIN_DF["detection_count"] = det_counts

## 列

In [ ]:
print(TRAIN_DF.info())
print()

### sequence_frame与video_frame

In [ ]:
eq_frames = TRAIN_DF[TRAIN_DF["video_frame"] == TRAIN_DF["sequence_frame"]]
uneq_frames = TRAIN_DF[TRAIN_DF["video_frame"] != TRAIN_DF["sequence_frame"]]
print("eq frames, uneq frames:", eq_frames.size, uneq_frames.size)
print()
#序列从0开始，可能在看不到海星的地方剪辑了视频
#视频被分割成序列，每个序列从0帧到n帧

## sequence_frames从0到N

In [ ]:
print("Sequence frames sequential and start from 0?")
for seq_name in TRAIN_DF["sequence"].unique():
    sequential = True
    numbers = TRAIN_DF[TRAIN_DF["sequence"] == seq_name]["sequence_frame"].values
    numbers.sort()
    
    i = 0
    for num in numbers:
        while i < num:
            print(f"Seq {seq_name}: {Fore.RED}Missing {i}{Fore.RESET}")
            i += 1
        i += 1

    if sequential:
        print(f"Seq {seq_name}: {Fore.GREEN}Yes{Fore.RESET}")
print()
# 可以  

### video_frames顺序

In [ ]:
print("Video frames sequential?")
for seq_name in TRAIN_DF["sequence"].unique():
    sequential = True
    numbers = TRAIN_DF[TRAIN_DF["sequence"] == seq_name]["video_frame"].values
    numbers.sort()
    
    i = numbers[0]
    for num in numbers:
        while i < num:
            print(f"Seq {seq_name}: {Fore.RED}Missing {i}{Fore.RESET}")
            i += 1
        i += 1

    if sequential:
        print(f"Seq {seq_name}: {Fore.GREEN}Yes{Fore.RESET}")
print()
#在评估时是否会给出顺序数据。
# image id是否只连接frame id和video id 
new_vid_ids = TRAIN_DF["video_id"].astype(str) + "-" + TRAIN_DF["video_frame"].astype(str)
print("How many images have strange image_ids:", (TRAIN_DF["image_id"] != new_vid_ids).sum())
# 是的

### 视频独特的序列名¶

In [ ]:
vid_seq_pairs = TRAIN_DF[["video_id", "sequence"]].drop_duplicates()
repeated_sequence_count = (vid_seq_pairs["sequence"].value_counts() != 1).sum()
print("How many repeated sequences:", repeated_sequence_count)
# 序列是唯一的

##注释的分发

In [ ]:
print(f"Starfish per image:", TRAIN_DF["detection_count"].value_counts())

bin_count = len(TRAIN_DF["detection_count"].unique())
plot = TRAIN_DF.hist(column="detection_count", figsize=(16,6), bins=bin_count)
ax = plot[0][0]
ax.set_title("Starfish count, per image")

TRAIN_DF_WITH_STARFISH = TRAIN_DF[TRAIN_DF["detection_count"] > 0]
bin_count = len(TRAIN_DF_WITH_STARFISH["detection_count"].unique())
plot = TRAIN_DF_WITH_STARFISH.hist(column="detection_count", figsize=(16,4), bins=bin_count)
ax = plot[0][0]
ax.set_title("Starfish count, per image (with 0 detections removed)");

## 在帧中检测如何改变的

In [ ]:
import math 


SEQUENCE_COUNT = len(TRAIN_DF["sequence"].drop_duplicates())
FIG_COLS = 3
FIG_ROWS = math.ceil(SEQUENCE_COUNT / FIG_COLS)
fig = plt.figure(figsize=(30, 30), constrained_layout=True)
# fig.tight_layout(pad=10.0)
# fig.tight_layout()

det_data = TRAIN_DF[["sequence", "video_id", "sequence_frame", "detection_count"]].drop_duplicates()
for i, seq_num in enumerate(det_data["sequence"].unique()):  # we know seq numbers are unique in train data
    #获取数据
    seq_data = det_data[det_data["sequence"] == seq_num].sort_values(by="sequence_frame")
    seq_data = seq_data.set_index(seq_data["sequence_frame"]).drop(columns="sequence_frame")
    video_id = seq_data["video_id"].iloc[0]
    
    #选择图形位置
    col = (i % FIG_COLS) + 1
    row = (i // FIG_COLS) + 1
    
    #细节
    ax = plt.subplot(FIG_ROWS, FIG_COLS, i+1)
    ax = seq_data["detection_count"].plot.line(ax=ax)
    ax.set_title(f"Video {video_id}, Sequence {seq_num}", fontsize=22)
    ax.yaxis.set_major_locator(plt.MaxNLocator(integer=True))
    ax.set_xlabel('Detections', fontsize=16)
    ax.set_ylabel('Sequence Frame', fontsize=16)
    


## 注释

In [ ]:
#缓存
video_ids = TRAIN_DF["video_id"].unique()
sel_video_id = 0
sel_video_df = TRAIN_DF[TRAIN_DF["video_id"] == sel_video_id]

sequences = sel_video_df["sequence"].unique()
sel_sequence = sequences[0]
sel_sequence_df = sel_video_df[sel_video_df["sequence"] == sel_sequence]

last_frame = sel_sequence_df["sequence_frame"].max()
sel_sequence_frame = 0
sel_sequence_frames = sel_sequence_df[sel_sequence_df["sequence_frame"] == sel_sequence_frame]
assert len(sel_sequence_frames) == 1
sel_video_frame = sel_sequence_frames["video_frame"].values[0]
sel_annotation = eval(sel_sequence_frames["annotations"].values[0])
  
#UI元素
dd_video_id = widgets.Dropdown(options=video_ids, description='Video ID:')
dd_sequence = widgets.Dropdown(options=sequences, description='Sequence:')
btn_first = widgets.Button(description="⏮️")
btn_back_50 = widgets.Button(description="⏪")
btn_back = widgets.Button(description="◀️")
btn_forward = widgets.Button(description="▶️")
btn_forward_50 = widgets.Button(description="⏩")
btn_last = widgets.Button(description="⏯")

out = widgets.Output()

dd_row = widgets.HBox([dd_video_id, dd_sequence])
btn_row = widgets.HBox([btn_first, btn_back_50, btn_back, btn_forward, btn_forward_50, btn_last])
all_widgets = widgets.VBox([dd_row, btn_row, out])

#选择助手-只更改数据
def set_frame(new_number):
    global sel_sequence_frame
    global sel_sequence_frames
    global sel_video_frame
    global sel_annotation
    
    sel_sequence_frame = max(0, min(new_number, last_frame))
    sel_sequence_frames = sel_sequence_df[sel_sequence_df["sequence_frame"] == sel_sequence_frame]
    assert len(sel_sequence_frames) == 1
    sel_video_frame = sel_sequence_frames["video_frame"].values[0]
    
    sel_annotation = eval(sel_sequence_frames["annotations"].values[0])

def set_sequence(new_number):
    global sel_sequence
    global sel_sequence_df
    global last_frame

    sel_sequence = new_number
    sel_sequence_df = sel_video_df[sel_video_df["sequence"] == sel_sequence]
    last_frame = sel_sequence_df["sequence_frame"].max()

    set_frame(0)
    
def set_video_id(new_id):
    global sel_video_id
    global sel_video_df
    global sequences
    
    sel_video_id = new_id
    sel_video_df = TRAIN_DF[TRAIN_DF["video_id"] == sel_video_id]

    sequences = sel_video_df["sequence"].unique()
    set_sequence(sequences[0])
    
#UI助手-处理UI事件+调用选择助手
def clear_output():
    out.clear_output()
    
def draw_image():
    img_path = ROOT_DIR / "train_images" / f"video_{sel_video_id}" / f"{sel_video_frame}.jpg"
    assert img_path.is_file(), f"Cannot find image {img_path}"
    cv_img = cv2.imread(str(img_path))
    cv_img = cv2.cvtColor(cv_img, cv2.COLOR_BGR2RGB)
    
    draw_bboxes(cv_img, sel_annotation)

    with out:
        print("Image", img_path)

        #找到一种方法来更新图像而不清除输出
        img_fig = plt.figure(figsize=(20,16), facecolor="#123456ff", frameon=False)
        img_ax = img_fig.add_subplot(1, 1, 1)
        img_ax.get_xaxis().set_visible(False)
        img_ax.get_yaxis().set_visible(False)
        img_ax.use_sticky_edges = False
        # img_ax.margins(x=0)  
        #用颜色透明代替作为一个hack。

        img_ax.imshow(cv_img)
        plt.show()

    
def draw_bboxes(cv_img, annotations):
    BBOX_COLOR_RGB = (255,126,0)
    for ann in annotations:
        cv2.rectangle(
            cv_img,
            (ann["x"] ,ann["y"]),
            (ann["x"] + ann["width"], ann["y"] + ann["height"]),
            color=BBOX_COLOR_RGB,
            thickness=3
        )

def on_click_forward(b):
    with out:
        set_frame(sel_sequence_frame + 1)
        clear_output()
        draw_image()
btn_forward.on_click(on_click_forward)
def on_click_back(b):
    with out:
        set_frame(sel_sequence_frame - 1)
        clear_output()
        draw_image()
btn_back.on_click(on_click_back)
def on_click_forward_50(b):
    with out:
        set_frame(sel_sequence_frame + 50)
        clear_output()
        draw_image()
btn_forward_50.on_click(on_click_forward_50)
def on_click_back_50(b):
    with out:
        set_frame(sel_sequence_frame - 50)
        clear_output()
        draw_image()
btn_back_50.on_click(on_click_back_50)
def on_click_first(b):
    with out:
        set_frame(0)
        clear_output()
        draw_image()
btn_first.on_click(on_click_first)
def on_click_last(b):
    with out:
        set_frame(last_frame)
        clear_output()
        draw_image()
btn_last.on_click(on_click_last)

def on_sequence_change(change):
    with out:
        if change["old"] == change["new"]:
            return
        set_sequence(change["new"])

        clear_output()
        draw_image()
dd_sequence.observe(on_sequence_change, names="value")

def on_video_id_change(change):
    with out:
        if change["old"] == change["new"]:
            return
        set_video_id(change["new"])
        dd_sequence.options = sequences
        
        clear_output()
        draw_image()
dd_video_id.observe(on_video_id_change, names="value")


display(all_widgets)
draw_image()